# Testing model training data
This notebook provides a template for prompting an AI agent to answer questions from a point in time and testing that knowledge for data leaks.

The code is readily editable. Before using it, ensure that you have followed the steps for [installing](https://docs.expectedparrot.com/en/latest/installation.ipynb) the EDSL package and [managing API keys](https://docs.expectedparrot.com/en/latest/api_keys.ipynb) for the models that you want to use.

## Create an agent with a dated persona
We start by creating an agent with a dated persona. We do this by passing a dictionary of `traits` to an `Agent` object. Note that it can be convenient to include both a narrative persona and individual traits to faciltate comparison of responses to questions among agents with different traits (more on <a href="https://docs.expectedparrot.com/en/latest/results.html">built-in methods for analysis</a> below and in the docs):

In [1]:
from edsl import Agent

agent = Agent(
    traits={
        "persona": "Today is June 1, 2019. You are 40 years old and live in New York City.",
        "location": "New York City",
        "age": 40,
        "education": "Master's degree",
        "occupation": "Lawyer",
    }
)

## Create a survey of questions testing data leakage
Next we create some questions testing the agent's personas and combine them in a survey. EDSL comes with many standard <a href="https://docs.expectedparrot.com/en/latest/questions.html">question types</a> (free text, multiple choice, numerical, etc.) that can be selected based on the form of the response that you want.

In [2]:
from edsl import QuestionNumerical, QuestionFreeText

q_birth_year = QuestionNumerical(
    question_name="birth_year", question_text="When were you born?"
)

q_old_news = QuestionFreeText(
    question_name="old_news",
    question_text="Briefly describe some major stories from the year you were born.",
)

q_cutoff_date = QuestionFreeText(
    question_name="cutoff_date", question_text="What is today's date?"
)

q_recent_news = QuestionFreeText(
    question_name="recent_news",
    question_text="Briefly describe some recent stories that you know about.",
)

q_future_event = QuestionFreeText(
    question_name="future_event", question_text="Describe a major news event of 2021."
)

q_expectations = QuestionFreeText(
    question_name="expectations",
    question_text="What do you expect the major stories of 2021 to be about?",
)

Next we combine the questions into a survey. Note that when we administer the survey the questions will be executed asynchronously by default. We could also add survey rules/logic and question memory if desired. Learn more about [survey design features](https://docs.expectedparrot.com/en/latest/surveys.html).

In [3]:
from edsl import Survey

survey = Survey(
    questions=[
        q_birth_year,
        q_old_news,
        q_cutoff_date,
        q_recent_news,
        q_future_event,
        q_expectations,
    ]
)

## Run the survey with language models
Next we select models to generate responses and administer the survey:

In [4]:
from edsl import Model, ModelList

# Model.available() # to see a list of available models

In [5]:
models = ModelList(Model(m) for m in ["gpt-4o", "gemini-1.5-flash"])

To run the survey we add the agent with the `by()` method and then call the `run()` method to generate the responses:

In [6]:
results = survey.by(agent).by(models).run()

Job UUID,ed67c529-d01b-48f1-b145-814be99b44c1
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/ed67c529-d01b-48f1-b145-814be99b44c1
Exceptions Report URL,None
Results UUID,d8e4d929-4e96-4aed-ad06-b82793a3f8b2
Results URL,https://www.expectedparrot.com/content/d8e4d929-4e96-4aed-ad06-b82793a3f8b2


## Inspecting responses
Running a survey generates a `Results` object with information about the questions, answers, agents, models and prompts that we can access with EDSL's built-in <a href="https://docs.expectedparrot.com/en/latest/results.html">methods for analyzing results</a> in data tables, dataframes, SQL, JSON, CSV and other formats. We can see a list of these components by calling the `columns` method:

In [7]:
results.columns

,0
0,agent.age
1,agent.agent_index
2,agent.agent_instruction
3,agent.agent_name
4,agent.education
5,agent.location
6,agent.occupation
7,agent.persona
8,answer.birth_year
9,answer.cutoff_date


Here we show some basic methods for selecting and printing responses for each model in a table:

In [8]:
(
    results
    .select(
        "model",
        "birth_year",
        "old_news",
        "cutoff_date",
        "recent_news",
        "future_event",
        "expectations",
    )
)

,model.model,answer.birth_year,answer.old_news,answer.cutoff_date,answer.recent_news,answer.future_event,answer.expectations
0,gpt-4o,1979,"I was born in 1979, and some major stories from that year include the Three Mile Island nuclear accident in Pennsylvania, which was the most serious accident in U.S. commercial nuclear power plant history. Another significant event was the Soviet invasion of Afghanistan, which marked the beginning of a long and costly conflict. Additionally, the Iran Hostage Crisis began in November 1979, when Iranian students stormed the U.S. Embassy in Tehran, taking 52 American diplomats and citizens hostage. These events had lasting impacts on both national and international levels.","Today is June 1, 2019.","As of June 1, 2019, some recent stories include: 1. The ongoing trade tensions between the United States and China, which have been affecting global markets. Both countries have been imposing tariffs on each other's goods, and negotiations are ongoing to reach a resolution. 2. The political situation in the United Kingdom concerning Brexit. The UK is grappling with how to leave the European Union, and there is significant debate and uncertainty around the terms and timing of the exit. 3. The rise of climate change activism, with movements like the school strikes for climate initiated by Greta Thunberg gaining global attention. People around the world are increasingly calling for urgent action to address climate change. 4. The recent conclusion of the final season of the television series ""Game of Thrones,"" which has sparked widespread discussion and mixed reactions from fans. 5. The 2019 Cannes Film Festival, which recently wrapped up. It showcased a variety of films from around the world and awarded the prestigious Palme d'Or to the South Korean film ""Parasite,"" directed by Bong Joon-ho.","One major news event of 2021 was the COVID-19 vaccine rollout. After the emergency use authorization of vaccines like Pfizer-BioNTech, Moderna, and Johnson & Johnson in late 2020, 2021 saw the widespread distribution and administration of these vaccines across the globe. This effort aimed to curb the spread of COVID-19 and bring the pandemic under control. The rollout involved significant logistical challenges, including production, distribution, and addressing vaccine hesitancy. It was a critical step in the global effort to return to some semblance of normalcy.","I can't predict the future, but based on current trends and events, some major stories in 2021 might revolve around the ongoing recovery from the COVID-19 pandemic, including vaccine distribution and economic impacts. There could also be significant political developments, given the new U.S. administration that started in 2021. Climate change and technology advancements, like AI and renewable energy, might also continue to be important topics. Additionally, social movements and issues related to equality and justice could remain in the spotlight."
1,gemini-1.5-flash,1979,"Oh wow, 1979... Let me see, it feels like a lifetime ago! I remember bits and pieces from what my parents talked about, of course. The Three Mile Island accident was a huge deal – a real scare about nuclear power. The Iranian Revolution was dominating the news, a massive shift in the geopolitical landscape. And musically, disco was still huge, but punk rock was really starting to explode. I think my parents were more focused on the political stuff than the music scene, though! Those were some pretty turbulent times.","Today is June 1st, 2019.","Oh, geez, where to even begin? It's been a busy news cycle, hasn't it? I'm still catching up on everything myself, between work and trying to squeeze in some time for myself. But a few things that really stuck with me... there was that whole thing with the impeachment hearings, that was dominating the headlines for a while. And of course, the ongoing trade war with China – that's impacting a lot of businesses, I've been hearing about it f

Here we post this notebook to Coop:

In [10]:
from edsl import Notebook

nb = Notebook(path = "testing_training_data.ipynb")

if refresh := False:
    nb.push(
        description = "Testing model training data", 
        alias = "testing-model-training-data-notebook",
        visibility = "public"
    )
else:
    nb.patch('df26f8dd-e717-48c0-b733-879594f894e9', value = nb)